# Homework 02

In [1]:
import csv
with open("recalls-truncated.csv") as f:
    reader = csv.reader(f)
    headers = next(reader)

In [4]:
with open("recalls-truncated.csv") as f:
    reader = csv.reader(f)
    headers = next(reader)
    data = [(row[1], int(row[4])) for row in reader if row[3] == 'Vehicle']
data

[('Triple E Recreational Vehicles', 341),
 ('Volvo Car USA, LLC', 74),
 ('Volkswagen Group of America, Inc.', 1042),
 ('Indian Motorcycle Company', 4653),
 ('Lion Electric Company', 13),
 ('Mercedes-Benz USA, LLC', 323963),
 ('Ford Motor Company', 101001),
 ('Mercedes-Benz USA, LLC', 123696),
 ('Shadow Trailer, LLC', 12),
 ('Honda (American Honda Motor Co.)', 19),
 ('Chrysler (FCA US, LLC)', 38),
 ('Kibbi, LLC', 292),
 ('REV Recreation Group', 1540),
 ('Volkswagen Group of America, Inc.', 37558),
 ('BMW of North America, LLC', 14086),
 ('New Flyer of America, Inc.', 746),
 ('BMW of North America, LLC', 4),
 ('Mercedes-Benz USA, LLC', 11),
 ('Hyundai Motor America', 853),
 ('Lightning eMotors Inc.', 27),
 ('Mercedes-Benz USA, LLC', 1),
 ('Mercedes-Benz USA, LLC', 69),
 ('PACCAR Incorporated', 2565),
 ('Mercedes-Benz USA, LLC', 278),
 ('PACCAR Incorporated', 15658),
 ('Airstream, Inc.', 1992),
 ('General Motors, LLC', 111242),
 ('Airstream, Inc.', 200),
 ('General Motors, LLC', 9),
 ('Ja

In [5]:
highest = max(data, key=lambda x: x[1])
lowest = min(data, key=lambda x: x[1])

In [6]:
print(lowest, highest)

('Chrysler (FCA US, LLC)', 1224078) ('Mercedes-Benz USA, LLC', 1)


In [1]:
import nelta as nt
import csv
class LabeledList:
    def __init__(self, data=None, index=None):
        if index is None:
            index = list(range(len(data)))
        self.values = data
        self.index = index

    def __str__(self):
        s = ''
        max_len = max([len(str(label)) for label in self.index])
        vals_max_len = max([len(str(v)) for v in self.values])
        format_spec = f'>{max_len}'
        for index, data in zip(self.index, self.values):
            s += f'{index:{format_spec}} {str(data):>{vals_max_len}}\n'
        return s

    def __iter__(self):
        return iter(self.values)

    def __repr__(self):
        return self.__str__()

    def __eq__(self, scalar):
        if self.index:
            return LabeledList([v == scalar for v in self.values], self.index)
        else:
            return LabeledList([v == scalar for v in self.values])

    def __ne__(self, scalar):
        if self.index:
            return LabeledList([v != scalar for v in self.values], self.index)
        else:
            return LabeledList([v != scalar for v in self.values])

    def __gt__(self, scalar):
        if self.index:
            return LabeledList([v > scalar if v is not None else False for v in self.values], self.index)
        else:
            return LabeledList([v > scalar if v is not None else False for v in self.values])

    def __lt__(self, scalar):
        if self.index:
            return LabeledList([v < scalar if v is not None else False for v in self.values], self.index)
        else:
            return LabeledList([v < scalar if v is not None else False for v in self.values])

    def map(self, f):
        values = [f(val) if val is not None else None for val in self.values]
        return LabeledList(values, self.index)

    def __getitem__(self, key_list):
        # try our best to make list of keys:

        # 1. first, if it's a labeled list, use the labeled list's 
        #    values property as the list of keys
        if isinstance(key_list, LabeledList):
            key_list = key_list.values

        # 2. at this point, if we still don't have a list, then assume we 
        #    have a non-sequence type, and wrap in a list
        if not isinstance(key_list, list):
            key_list = [key_list]

        # now that we definitely have a list... check if it's only
        # booleans; one way to do this is filter for only boolean values
        # and check the length against the original unfiltered list
        if len([v for v in key_list if type(v) is bool]) == len(key_list):
            # we have a bunch of booleans, keep only the values that
            # have the same label as the labels that have true
            return self.__filter(key_list) 
        else:
            # we have a bunch of keys... so get all values with a matching
            # label / key
            index = [] # labels for returned LabeledList
            data = []  # values for returned LabeledList
            for key in key_list: 
                # find key matches, and get back both the label and value
                for label, val in self.__find(key):
                    index.append(label)
                    data.append(val)
            return data[0] if len(data) == 1 else LabeledList(data, index)

    def __filter(self, filter_list):
        """ given a list of booleans, only give back the values
        that align with True as a LabeledList
        """
        index = []
        data = []
        if len(filter_list) != len(self.index):
            raise IndexError('Length of indexes does not match length of boolean list')
        for i, include in enumerate(filter_list):
            if include:
                index.append(self.index[i])
                data.append(self.values[i])
        return LabeledList(data, index)
    
    def __find(self, k):
        """give back all labels and values based on key
        """
        index, data = [], []
        matches = [(label, self.values[i]) for i, label in enumerate(self.index) if k == label]
        if len(matches) == 0:
            raise KeyError(f'Index label not found {k}')
        return matches


    
class Table:
    # implement your table class here
    def __init__(self, data, index=None, columns=None):
        self.data = data
        self.index = list(range(len(data))) if index is None else index
        self.columns = list(range(len(data[0]))) if columns is None else columns

    def __str__(self):
        table = []
        table.append(["{:^10}".format("     ")] + ["{:^10}".format(c) for c in self.columns])
        for i, row in enumerate(self.data):
            table.append(["{:^10}".format(self.index[i])] + ["{:^10}".format(r) for r in row])
        return "\n".join(["".join(r) for r in table])

    def __repr__(self):
        return self.__str__()

    def __getitem__(self, key):
        if isinstance(key, list):
            if type(key[0]) in [bool, True, False]:
                data = [self.data[i] for i in range(len(self.data)) if key[i]]
                index = [self.index[i] for i in range(len(self.data)) if key[i]]
                return Table(data, index, self.columns)
            else:
                columns = key
                data = [[row[self.columns.index(col)] for col in key] for row in self.data]
                return Table(data, self.index, columns)
        else:
            column_indices = [i for i, col in enumerate(self.columns) if col == key]
            if len(column_indices) == 0:
                raise KeyError(f'Column not found: {key}')
            elif len(column_indices) == 1:
                return LabeledList([row[column_indices[0]] for row in self.data], self.index)
            else:
                data = [[row[i] for i in column_indices] for row in self.data]
                columns = [self.columns[i] for i in column_indices]
                return Table(data, self.index, columns)

    def head(self, n):
        data = self.data[:n]
        index = self.index[:n]
        return Table(data, index, self.columns)

    def tail(self, n):
        data = self.data[-n:]
        index = self.index[-n:]
        return Table(data, index, self.columns)

    def shape(self):
        return (len(self.data), len(self.columns))



def read_csv(fn):
    data = []
    with open(fn, "r") as f:
        reader = csv.reader(f)
        header = next(reader)
        columns = header
        for i, row in enumerate(reader):
            if not all(x == "" for x in row):
                data_row = []
                for item in row:
                    try:
                        data_row.append(float(item))
                    except ValueError:
                        data_row.append(item)
                data.append(data_row)
    index = list(range(len(data)))
    return Table(data, index, columns)

In [2]:
t = nt.read_csv('recalls-truncated.csv')
t.shape()

(350, 5)

In [3]:
t.head(0)

             Date   Manufacturer Subject  Recall TypePotentially Affected

In [4]:
t.head(4)

             Date   Manufacturer Subject  Recall TypePotentially Affected
    0     01/06/2023Triple E Recreational VehiclesBattery Disconnect Switch May Short Vehicle    341.0   
    1     01/05/2023Volvo Car USA, LLCSteering Wheel May Lock Up Vehicle     74.0   
    2     12/29/2022Volkswagen Group of America, Inc.12-Volt Battery Cable May Short Circuit Vehicle    1042.0  
    3     12/29/2022Indian Motorcycle CompanyKickstand May Not Retract Properly/FMVSS 123 Vehicle    4653.0  

In [6]:
last_four = t.tail(4)
last_four

             Date   Manufacturer Subject  Recall TypePotentially Affected
   346    08/25/2022Rivian Automotive, LLCImproperly Secured Front Seat Belt Anchor Vehicle    207.0   
   347    08/25/2022Chrysler (FCA US, LLC)Rearview Camera Image May Not Display/FMVSS 111 Vehicle    7895.0  
   348    08/25/2022General Motors, LLCSecondary Hood Latch Corrosion Vehicle   120688.0 
   349    08/25/2022PT. Elangperdana Tyre IndustryTire Sidewall Separation/ FMVSS 139   Tire    31500.0  

In [16]:
ll = LabeledList(last_four['Subject'], [346 + i for i in range(4)])
ll

346       Improperly Secured Front Seat Belt Anchor
347 Rearview Camera Image May Not Display/FMVSS 111
348                  Secondary Hood Latch Corrosion
349             Tire Sidewall Separation/ FMVSS 139

In [15]:
for val in ll:
    print(val)

Improperly Secured Front Seat Belt Anchor
Rearview Camera Image May Not Display/FMVSS 111
Secondary Hood Latch Corrosion
Tire Sidewall Separation/ FMVSS 139


In [17]:
last_four[['Manufacturer', 'Subject']]

          Manufacturer Subject  
   346    Rivian Automotive, LLCImproperly Secured Front Seat Belt Anchor
   347    Chrysler (FCA US, LLC)Rearview Camera Image May Not Display/FMVSS 111
   348    General Motors, LLCSecondary Hood Latch Corrosion
   349    PT. Elangperdana Tyre IndustryTire Sidewall Separation/ FMVSS 139

In [36]:
vehicles = t[t["Recall Type"] == "Vehicle" ]
vehicles

             Date   Manufacturer Subject  Recall TypePotentially Affected
    0     01/06/2023Triple E Recreational VehiclesBattery Disconnect Switch May Short Vehicle    341.0   
    1     01/05/2023Volvo Car USA, LLCSteering Wheel May Lock Up Vehicle     74.0   
    2     12/29/2022Volkswagen Group of America, Inc.12-Volt Battery Cable May Short Circuit Vehicle    1042.0  
    3     12/29/2022Indian Motorcycle CompanyKickstand May Not Retract Properly/FMVSS 123 Vehicle    4653.0  
    4     12/29/2022Hendrickson USA. L.L.C.Loose Brake Pad Retaining Clip May Damage TireEquipment    12.0   
    5     12/23/2022Lion Electric CompanyAdjusting Ring Fasteners Not Tightened Properly Vehicle     13.0   
    6     12/23/2022WARN Industries, Inc.Corrosion on Remote Control Circuit BoardEquipment  121187.0 
    7     12/23/2022Mercedes-Benz USA, LLCEngine Stall from Water Intrusion into Vehicle Vehicle   323963.0 
    8     12/23/2022Ford Motor CompanySeat Belt Warning System Malfunction/FMVSS 